In [1]:
import pandas as pd  # For data manipulation and reading CSV files
import numpy as np  # For numerical computations
from sklearn.linear_model import LogisticRegression  # Logistic Regression model
from sklearn.model_selection import cross_val_score  # Cross-validation
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Evaluation metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, confusion_matrix

train_data = pd.read_csv("train_data.csv")

print(train_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11067 entries, 0 to 11066
Columns: 167 entries, id to away_pitcher_wpa_def_skew
dtypes: bool(1), float64(157), int64(1), object(8)
memory usage: 14.0+ MB
None


In [2]:
# Drop rows with missing values in the specified columns


#fill Nan
numerical_columns = train_data.select_dtypes(include=['number']).columns  # Select only numeric columns
train_data[numerical_columns] = train_data[numerical_columns].fillna(train_data[numerical_columns].mean())
#fill train_data
new_columns = pd.DataFrame({
    'team_rest_diff': train_data['home_team_rest'] - train_data['away_team_rest'],
    'pitcher_rest_diff': train_data['home_pitcher_rest'] - train_data['away_pitcher_rest'],
    'batting_avg_diff': train_data['home_batting_batting_avg_10RA'] - train_data['away_batting_batting_avg_10RA'],
    'pitching_ERA_diff': train_data['home_pitching_earned_run_avg_10RA'] - train_data['away_pitching_earned_run_avg_10RA'],
    'onbase_perc' : train_data['home_batting_onbase_plus_slugging_10RA'] / train_data['away_batting_onbase_plus_slugging_10RA']
})

# Concatenate the new columns to the original DataFrame
train_data = pd.concat([train_data, new_columns], axis=1)
columns_to_drop = ['away_pitcher', 'home_pitcher', 'date','id', 'home_team_abbr','away_team_abbr','season','home_pitcher_rest','away_pitcher_rest','home_team_rest','away_team_rest','home_batting_batting_avg_10RA','away_batting_batting_avg_10RA','home_pitching_earned_run_avg_10RA','away_pitching_earned_run_avg_10RA','home_batting_onbase_plus_slugging_10RA','away_batting_onbase_plus_slugging_10RA']
train_data.drop(columns=columns_to_drop, inplace=True)
train_data['is_night_game'] = train_data['is_night_game'].apply(
    lambda x: np.random.choice([True, False]) if pd.isnull(x) else x
)

# Example of splitting the dataset by winning and losing teams
# Assuming 'home_team_win' is the column that indicates the outcome (1 = win, 0 = loss)
winning_team_data = train_data[train_data['home_team_win'] == 1]
losing_team_data = train_data[train_data['home_team_win'] == 0]
numeric_columns = train_data.select_dtypes(include=['number']).columns
# Loop through all columns and perform a t-test for each column
count = 0
add_colums=[]
for column in numeric_columns:
    if column != 'home_team_win':  # Skip the outcome column itself
        # Perform t-test (assuming data is numeric)
        t_stat, p_value = stats.ttest_ind(winning_team_data[column], losing_team_data[column], nan_policy='omit')
        
        # If p-value is less than 0.05, print the result
        if p_value < 0.05:
            # print(f"Significant variable: {column} | p-value: {p_value}")
            add_colums.append(column)
# One-hot encode 'home_team_season' and 'away_team_season'
original_columns = train_data.columns.tolist()

# One-hot encode 'home_team_season' and 'away_team_season'
season_columns = ['home_team_season', 'away_team_season']
train_data = pd.get_dummies(train_data, columns=season_columns, drop_first=True)

# Save the new column names after one-hot encoding
new_columns = train_data.columns.tolist()

# Identify the newly added columns
dummy_columns = [col for col in new_columns if col not in original_columns]

# Add these to your `add_columns` list
add_colums.extend(dummy_columns)

# View the resulting columns
print(train_data.columns)

Index(['is_night_game', 'home_team_win', 'home_batting_onbase_perc_10RA',
       'home_batting_leverage_index_avg_10RA', 'home_batting_RBI_10RA',
       'away_batting_onbase_perc_10RA', 'away_batting_leverage_index_avg_10RA',
       'away_batting_RBI_10RA', 'home_pitching_SO_batters_faced_10RA',
       'home_pitching_H_batters_faced_10RA',
       ...
       'away_team_season_TOR_2021', 'away_team_season_TOR_2022',
       'away_team_season_TOR_2023', 'away_team_season_WSN_2016',
       'away_team_season_WSN_2017', 'away_team_season_WSN_2018',
       'away_team_season_WSN_2019', 'away_team_season_WSN_2021',
       'away_team_season_WSN_2022', 'away_team_season_WSN_2023'],
      dtype='object', length=571)


In [ ]:

# std_columns = [col for col in add_colums if 'std' in col]
# skew_columns = [col for col in add_colums if 'skew' in col]
# selected_columns = skew_columns
# Filter columns containing 'mean'

y = train_data['home_team_win']
X = train_data[add_colums]
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 創建模型
model = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)#low C simplifies the model too much
model.fit(X_train, y_train)

# 進行交叉驗證 (5折)
cv_scores = cross_val_score(model, X_train, y_train, cv=5)

# 交叉驗證結果
print(f"Cross-validation Accuracy: {cv_scores.mean():.4f}")
print(cv_scores)

y_val = model.predict(X_train)
y_pred = model.predict(X_test)

# Train accuracy
accuracy = accuracy_score(y_train, y_val)
print(f"Train Accuracy: {accuracy:.4f}")

# Confusion matrix for train data
cm = confusion_matrix(y_train, y_val)
print(f"Confusion Matrix:\n{cm}")

# Test accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Confusion matrix for test data
cm = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{cm}")

NameError: name 'train_data' is not defined

In [206]:
print(train_data.columns.tolist())
std_columns = [col for col in train_data.columns if 'std' in col]
skew_columns = [col for col in train_data.columns if 'skew' in col]
selected_columns = std_columns + skew_columns
y = train_data['home_team_win']
X = train_data.drop(columns='home_team_win')
X.drop(columns=selected_columns)

print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 創建模型
model = LogisticRegression(penalty='l2',solver='liblinear', max_iter=2000, random_state=42)#low C simplifies the model too much #C= 1
model.fit(X_train, y_train)

# 進行交叉驗證 (5折)
cv_scores = cross_val_score(model, X_train, y_train, cv=5)

# 交叉驗證結果
print(f"Cross-validation Accuracy: {cv_scores.mean():.4f}")
print(cv_scores)

y_val = model.predict(X_train)
y_pred = model.predict(X_test)

# Train accuracy
accuracy = accuracy_score(y_train, y_val)
print(f"Train Accuracy: {accuracy:.4f}")

# Confusion matrix for train data
cm = confusion_matrix(y_train, y_val)
print(f"Confusion Matrix:\n{cm}")

# Test accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Confusion matrix for test data
cm = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{cm}")

['is_night_game', 'home_team_win', 'home_batting_onbase_perc_10RA', 'home_batting_onbase_plus_slugging_10RA', 'home_batting_leverage_index_avg_10RA', 'home_batting_RBI_10RA', 'away_batting_onbase_perc_10RA', 'away_batting_onbase_plus_slugging_10RA', 'away_batting_leverage_index_avg_10RA', 'away_batting_RBI_10RA', 'home_pitching_SO_batters_faced_10RA', 'home_pitching_H_batters_faced_10RA', 'home_pitching_BB_batters_faced_10RA', 'away_pitching_SO_batters_faced_10RA', 'away_pitching_H_batters_faced_10RA', 'away_pitching_BB_batters_faced_10RA', 'home_pitcher_earned_run_avg_10RA', 'home_pitcher_SO_batters_faced_10RA', 'home_pitcher_H_batters_faced_10RA', 'home_pitcher_BB_batters_faced_10RA', 'away_pitcher_earned_run_avg_10RA', 'away_pitcher_SO_batters_faced_10RA', 'away_pitcher_H_batters_faced_10RA', 'away_pitcher_BB_batters_faced_10RA', 'home_team_errors_mean', 'home_team_errors_std', 'home_team_errors_skew', 'away_team_errors_mean', 'away_team_errors_std', 'away_team_errors_skew', 'home_t

In [199]:
print(train_data.columns.tolist())
std_columns = [col for col in train_data.columns if 'std' in col]
skew_columns = [col for col in train_data.columns if 'skew' in col]
selected_columns = std_columns + skew_columns
y = train_data['home_team_win']
X = train_data.drop(columns='home_team_win')
X.drop(columns=selected_columns)
print(X.shape)
print(y.shape)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 創建模型
model = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)#low C simplifies the model too much
model.fit(X, y)

# 進行交叉驗證 (5折)
cv_scores = cross_val_score(model, X, y, cv=5)

# 交叉驗證結果
print(f"Cross-validation Accuracy: {cv_scores.mean():.4f}")
print(cv_scores)

y_val = model.predict(X)

# Train accuracy
accuracy = accuracy_score(y, y_val)
print(f"Train Accuracy: {accuracy:.4f}")

# Confusion matrix for train data
cm = confusion_matrix(y, y_val)
print(f"Confusion Matrix:\n{cm}")


['is_night_game', 'home_team_win', 'home_batting_onbase_perc_10RA', 'home_batting_onbase_plus_slugging_10RA', 'home_batting_leverage_index_avg_10RA', 'home_batting_RBI_10RA', 'away_batting_onbase_perc_10RA', 'away_batting_onbase_plus_slugging_10RA', 'away_batting_leverage_index_avg_10RA', 'away_batting_RBI_10RA', 'home_pitching_SO_batters_faced_10RA', 'home_pitching_H_batters_faced_10RA', 'home_pitching_BB_batters_faced_10RA', 'away_pitching_SO_batters_faced_10RA', 'away_pitching_H_batters_faced_10RA', 'away_pitching_BB_batters_faced_10RA', 'home_pitcher_earned_run_avg_10RA', 'home_pitcher_SO_batters_faced_10RA', 'home_pitcher_H_batters_faced_10RA', 'home_pitcher_BB_batters_faced_10RA', 'away_pitcher_earned_run_avg_10RA', 'away_pitcher_SO_batters_faced_10RA', 'away_pitcher_H_batters_faced_10RA', 'away_pitcher_BB_batters_faced_10RA', 'home_team_errors_mean', 'home_team_errors_std', 'home_team_errors_skew', 'away_team_errors_mean', 'away_team_errors_std', 'away_team_errors_skew', 'home_t

In [203]:
testing_data = pd.read_csv("same_season_test_data.csv")
# Drop rows with missing values in the specified columns
#fill train_data

new_columns = pd.DataFrame({
    'team_rest_diff': testing_data['home_team_rest'] - testing_data['away_team_rest'],
    'pitcher_rest_diff': testing_data['home_pitcher_rest'] - testing_data['away_pitcher_rest'],
    'batting_avg_diff': testing_data['home_batting_batting_avg_10RA'] - testing_data['away_batting_batting_avg_10RA'],
    'pitching_ERA_diff': testing_data['home_pitching_earned_run_avg_10RA'] - testing_data['away_pitching_earned_run_avg_10RA']
})
testing_data = pd.concat([testing_data, new_columns], axis=1)
columns_to_drop = ['away_pitcher', 'home_pitcher','id', 'home_team_abbr','away_team_abbr','season','home_pitcher_rest','away_pitcher_rest','home_team_rest','away_team_rest','home_batting_batting_avg_10RA','away_batting_batting_avg_10RA','home_pitching_earned_run_avg_10RA','away_pitching_earned_run_avg_10RA']
testing_data.drop(columns=columns_to_drop, inplace=True)

numerical_columns = testing_data.select_dtypes(include=['number']).columns  # Select only numeric columns
testing_data[numerical_columns] = testing_data[numerical_columns].fillna(testing_data[numerical_columns].mean())
testing_data['is_night_game'] = testing_data['is_night_game'].apply(
    lambda x: np.random.choice([True, False]) if pd.isnull(x) else x
)
testing_data['is_night_game'] = testing_data['is_night_game'].astype(float)
# One-hot encode 'home_team_season' and 'away_team_season'
season_columns = ['home_team_season', 'away_team_season']
testing_data = pd.get_dummies(testing_data, columns=season_columns, drop_first=True)
print(testing_data.columns.tolist())
std_columns = [col for col in testing_data.columns if 'std' in col]
skew_columns = [col for col in testing_data.columns if 'skew' in col]
selected_columns = std_columns + skew_columns
testing_data.drop(columns=selected_columns)
test_labels = model.predict(testing_data)
ids = pd.read_csv("same_season_test_data.csv")['id']

# Create the output DataFrame with 'id' and 'test_labels'
output_data = pd.DataFrame({'id': ids, 'home_team_win': test_labels})

# Save the DataFrame to a CSV file
output_data.to_csv('output_file.csv', index=False)

print("File saved as 'output_file.csv'.")


['is_night_game', 'home_batting_onbase_perc_10RA', 'home_batting_onbase_plus_slugging_10RA', 'home_batting_leverage_index_avg_10RA', 'home_batting_RBI_10RA', 'away_batting_onbase_perc_10RA', 'away_batting_onbase_plus_slugging_10RA', 'away_batting_leverage_index_avg_10RA', 'away_batting_RBI_10RA', 'home_pitching_SO_batters_faced_10RA', 'home_pitching_H_batters_faced_10RA', 'home_pitching_BB_batters_faced_10RA', 'away_pitching_SO_batters_faced_10RA', 'away_pitching_H_batters_faced_10RA', 'away_pitching_BB_batters_faced_10RA', 'home_pitcher_earned_run_avg_10RA', 'home_pitcher_SO_batters_faced_10RA', 'home_pitcher_H_batters_faced_10RA', 'home_pitcher_BB_batters_faced_10RA', 'away_pitcher_earned_run_avg_10RA', 'away_pitcher_SO_batters_faced_10RA', 'away_pitcher_H_batters_faced_10RA', 'away_pitcher_BB_batters_faced_10RA', 'home_team_errors_mean', 'home_team_errors_std', 'home_team_errors_skew', 'away_team_errors_mean', 'away_team_errors_std', 'away_team_errors_skew', 'home_team_spread_mean',

ValueError: data type <class 'numpy.object_'> not inexact